# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
driver.find_element_by_id('checkbox').click()

In [5]:
driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]').click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [6]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [7]:
dropdown = Select(driver.find_element_by_id('slcJurisdiction'))
dropdown.select_by_visible_text('Statewide')

In [8]:
driver.find_element_by_id('txtTradeName').send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [9]:
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [10]:
titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    name = title.find_elements_by_tag_name('td')[1].text
    print(name)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [11]:
titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    try:
        detail_url = title.find_elements_by_tag_name('td')[-1].find_element_by_tag_name('a').get_attribute('href')
        print(detail_url)
    except:
        print("We don't have a url for this one.")

https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2RfKmQ%2FBrww%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=9C29FZTW3bA%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WV3x0tF0Zfo%3D
We don't have a url for this one.
We don't have a url for this one.


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

```Python
titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    detail_url = title.find_elements_by_tag_name('td')[2].find_element_by_tag_name('a').get_attribute('href')
    driver.get(detail_url)
    # pretend to be scraping here...
    back = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[4]/tbody/tr/td[1]/a')
    driver.execute_script('arguments[0].scrollIntoView(true)', back)
    back.click()
```
The above code throws a Stale Element Reference Exception.

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [12]:
detail_urls = []
titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    row = {}
    name = title.find_elements_by_tag_name('td')[1].text
    row['name'] = name
    try:
        detail_url = title.find_elements_by_tag_name('td')[-1].find_element_by_tag_name('a').get_attribute('href')
        row['url'] = detail_url
    except:
        pass
    detail_urls.append(row)

In [13]:
df = pd.DataFrame(detail_urls)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [14]:
titles = driver.find_elements_by_class_name('searchfieldtitle')
for title in titles:
    name = title.find_elements_by_tag_name('td')[1].text
    try:
        detail_url = title.find_elements_by_tag_name('td')[-1].find_element_by_tag_name('a').get_attribute('href')
    except:
        pass
# find_element_by_xpath looks a mess, while find_element_by_partial_link_text looks readable and friendly
# button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
button = driver.find_element_by_partial_link_text('Next')
driver.execute_script('arguments[0].scrollIntoView(true)', button)
button.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [15]:
# Renavigating to the original result page

# driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
# dropdown = Select(driver.find_element_by_id('slcJurisdiction'))
# dropdown.select_by_visible_text('Statewide')
# driver.find_element_by_id('txtTradeName').send_keys('VAP%')
# driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').submit()

```Python
while True:
    titles = driver.find_elements_by_class_name('searchfieldtitle')
    for title in titles:
        name = title.find_elements_by_tag_name('td')[1].text
        try:
            detail_url = title.find_elements_by_tag_name('td')[-1].find_element_by_tag_name('a').get_attribute('href')
        except:
            pass
    print('Scraping a new page')
    button = driver.find_element_by_partial_link_text('Next')
    driver.execute_script('arguments[0].scrollIntoView(true)', button)
    button.click()
```

The above code works but will get an error on the last page since it can't locate a 'Next' button.

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [16]:
# Renavigating to the original result page

driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]').submit()

In [17]:
detail_urls = []
try:
    while True:
        titles = driver.find_elements_by_class_name('searchfieldtitle')
        for title in titles:
            row = {}
            name = title.find_elements_by_tag_name('td')[1].text
            row['name'] = name
            try:
                detail_url = title.find_elements_by_tag_name('td')[-1].find_element_by_tag_name('a').get_attribute('href')
                row['url'] = detail_url
            except:
                pass
            detail_urls.append(row)
        button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script('arguments[0].scrollIntoView(true)', button)
        button.click()
except:
    pass

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [18]:
df = pd.DataFrame(detail_urls)
df.to_csv('vape-shops-basic.csv', index=False)

# Okay, let’s scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [19]:
pd.set_option('display.max_colwidth', -1)
df.head(1)

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2RfKmQ%2FBrww%3D


### Use Selenium to visit that page

In [20]:
driver.get(df['url'][0])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [21]:
mailing = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
print(mailing)
print('-------')
location = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
print(location)
print('-------')
license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
print(license_info)
print('-------')
paid = driver.find_elements_by_class_name('rt')[-1].text
print(paid)
print('-------')
issuedby = driver.find_elements_by_class_name('tablecelltext')[-2:]
for line in issuedby:
    print(line.text)

AMIN NARGIS
1104 PLANTERS PLACE
SALISBURY, MD 21804
-------
VAPE IT STORE II
1015 S SALISBURY BLVD
SALISBURY, MD 21801
-------
License Status: Issued
License No.: 22375606
Control No.: 22884439
Date of Issue: 4/27/2018
Months Paid: 12
Exp. Date: 4/30/2019
Subdivision: 22 Salisbury
-------
$ 41.00
-------
WICOMICO COUNTY, CLERK OF CIRCUIT COURT
P.O. BOX 198
SALISBURY, MARYLAND 21803-0198
 


### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [22]:
driver.get(df['url'][0])

details = {}
mailing = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
details['mailing_address'] = mailing
location = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
details['location_address'] = location
license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
details['license_info'] = license_info
paid = driver.find_elements_by_class_name('rt')[-1].text
details['amount_paid'] = paid
issuedby = driver.find_elements_by_tag_name('tr')[-7].text
details['issued_by'] = issuedby

details

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'amount_paid': '$ 41.00',
 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [23]:
def get_details(row):
    try:
        driver.get(row['url'])
        details = {}
        mailing = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
        details['mailing_address'] = mailing
        location = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
        details['location_address'] = location
        license_info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
        details['license_info'] = license_info
        paid = driver.find_elements_by_class_name('rt')[-1].text
        details['amount_paid'] = paid
        issuedby = driver.find_elements_by_tag_name('tr')[-7].text
        details['issued_by'] = issuedby
        return pd.Series(details)
    except:
        pass

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [24]:
extended_df = df.apply(get_details, axis=1).join(df)

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [25]:
extended_df.to_csv('vape-total.csv', index=False)

In [26]:
pd.read_csv('vape-total.csv')

,mailing_address,location_address,license_info,amount_paid,issued_by,name,url
0,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2RfKmQ%2FBrww%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=9C29FZTW3bA%3D
2,"ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WV3x0tF0Zfo%3D
3,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
4,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
5,DISBROW II EMERSON HARRINGTON,"VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037",License Status: Issued\nLicense No.: 02310799\nControl No.: 02686069\nDate of Issue: 4/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=MmTpaQrpllw%3D
6,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",License Status: Issued\nLicense No.: 13343011\nControl No.: 13856368\nDate of Issue: 4/30/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 13 Howard County,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045",VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=u7mFPwy9isU%3D
7,FAIRGROUND VILLAGE LLC,"VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157",License Status: Issued\nLicense No.: 06327188\nControl No.: 06946760\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Westminster,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155",VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=TfOlEnzUR7s%3D
8,NaN,NaN,NaN,NaN,NaN,VAPE HAVEN,NaN
9,NaN,NaN,NaN,NaN,NaN,VAPORS LOUNGE,NaN
